In [ ]:
from matplotlib import rcParams, rc
rcParams.update({'figure.autolayout': True})

import csv
import torch
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import math
import pandas as pd

import sys
import os

from Tarjet import Tarjet

from Phisicsparams import PhysicsParams
from utils import *
from MTMM import *
from tqdm import tqdm

from multiresglonet import GLOnet
from material_database import MatDatabase

ModuleNotFoundError: No module named 'Tarjet'

In [9]:
params = Params()

NameError: name 'Params' is not defined

In [ ]:
params.thickness_sup = 0.2
params.N_layers = 40
#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
params.materials = ['Al2O3','TiO2', 'SiO2']
params.user_define = True
if params.user_define:
  params.n_min = 1.00
  params.n_max = 2.5
  params.M_discretion_n = 300
  params.M_materials = params.M_discretion_n
  params.n_database = torch.tensor(np.array([np.linspace(params.n_min,params.n_max,params.M_discretion_n)]))
else:
  pass # definirlo en funcion de los parametros fisicos (physics_params)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

params.alpha_sup = 20
params.numIter = 200
params.sigma = 0.045
params.batch_size = 200
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
params.net = 'Res'
params.res_layers = 20                                                                             # Cantidad de bloques Residuales del bloque ResNet
params.res_dim = 512                                                                              # Cantidad de neuronas en la capa de entrada al bloque ResNet
params.noise_dim = 20                                                                              # Dimension de la Capa de entrada
params.lr = 0.043                                                                                   # Tasa de aprendizaje del optimizador Adam (learning rate)
params.beta1 = 0.9                                                                                 # Coeficiente de decaimiento para el momento del primer orden del optimizador Adam
params.beta2 = 0.99                                                                                # Coeficiente de decaimiento para el momento del segundo orden del optimizador Adam
params.weight_decay = 0.001                                                                        # Termino de decaimiento del peso para regularizar los pesos del generador durante la optimizacion
params.step_size = 40000                                                                           # Numero de epicas despues de las cuales se reduce la tasa de aprendizaje
params.gamma = 0.5                                                                                 # El factor de reduccion para la tasa de aprendizaje. Despues de cada step_size epocas, la tasa de aprendizaje se multiplica por gamma


In [ ]:
params.condiciones = 3
physicsparams = PhysicsParams(params.condiciones, user_define=True)

n_interna = 1.2

physicsparams.n_bot_1 = n_interna
physicsparams.n_top_1 = 1
physicsparams.k_1 = 370
physicsparams.k_values.append(physicsparams.k_1)
physicsparams.theta_1 = 45
physicsparams.lambda_min_1 = 380
physicsparams.lambda_max_1 = 750
physicsparams.pol_1 = "TE"

physicsparams.n_bot_2 = 1
physicsparams.n_top_2 = n_interna
physicsparams.k_2 = 190
physicsparams.k_values.append(physicsparams.k_2)
physicsparams.theta_2 = 45
physicsparams.lambda_min_2 = 380
physicsparams.lambda_max_2 = 570
physicsparams.pol_2 = "TE"

physicsparams.n_bot_3 = 1
physicsparams.n_top_3 = n_interna
physicsparams.k_3 = 400
physicsparams.k_values.append(physicsparams.k_3)
physicsparams.theta_3 = 0
physicsparams.lambda_min_3 = 380
physicsparams.lambda_max_3 = 495
physicsparams.pol_3 = "TE"


physicsparams.generate_physics_params()
physicsparams.view_attributes()

tarjet = Tarjet(params.condiciones, physicsparams.k_values, params.user_define)
tarjet.configure_targets()

if not params.user_define:
    params.matdatabase =  MatDatabase(params.materials)
    params.n_database = params.matdatabase.interp_wv(2 * math.pi/physicsparams.ks, params.materials, True)
    params.M_materials =  params.n_database.size(0)
else:
    pass


In [ ]:
tarjet.tarjets_transmisions["tarjet_1"].view(-1)[0:190 ] = 1
tarjet.tarjets_reflections["tarjet_1"].view(-1)[190:] = 1

tarjet.tarjets_transmisions["tarjet_2"].view(-1)[0:115 ] = 1
tarjet.tarjets_reflections["tarjet_2"].view(-1)[115:] = 1

tarjet.tarjets_transmisions["tarjet_3"].view(-1)[:] = 1
tarjet.tarjets_reflections["tarjet_3"].view(-1)[:] = 0

In [ ]:
# Subplots para Transmisiones
fig, axs = plt.subplots(1, 3, figsize=(13, 3))  # 1 fila, 3 columnas
fig.subplots_adjust(wspace=0.4)
xlim = (300, 800)
ylim = (-0.5, 1.5)
vertical_lines = [380, 495, 570, 750]

for i, ax in enumerate(axs):
    if i == 0:
        ax.plot(2*math.pi / physicsparams.k_1 * 1000, tarjet.tarjets_transmisions["tarjet_1"].view(-1), ".-", color="red", label="Transmision_Tarjet_1")
        ax.set_title("Transmision Tarjet 1", fontsize=20)
    elif i == 1:
        ax.plot(2*math.pi / physicsparams.k_2 * 1000, tarjet.tarjets_transmisions["tarjet_2"].view(-1), ".-", color="green", label="Transmision_Tarjet_2")
        ax.set_title("Transmision Tarjet 2", fontsize=20)
    elif i == 2:
        ax.plot(2*math.pi / physicsparams.k_3 * 1000, tarjet.tarjets_transmisions["tarjet_3"].view(-1), ".-", color="blue", label="Transmision_Tarjet_3")
        ax.set_title("Transmision Tarjet 3", fontsize=20)

    ax.set_xlabel("Wavelength (nm)", fontsize=16)
    ax.set_ylabel("Intensity", fontsize=16)
    ax.legend(fontsize=10)
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # Agregar líneas verticales punteadas
    for line_pos in vertical_lines:
        ax.axvline(x=line_pos, linestyle='--', color='gray')

# Mostrar la figura de transmisiones
plt.show()

# Subplots para Reflexiones
fig, axs = plt.subplots(1, 3, figsize=(13, 3))  # 1 fila, 3 columnas
fig.subplots_adjust(wspace=0.4)
xlim = (300, 800)
ylim = (-0.5, 1.5)
vertical_lines = [380, 495, 570, 750]

for i, ax in enumerate(axs):
    if i == 0:
        ax.plot(2*math.pi / physicsparams.k_1 * 1000, tarjet.tarjets_reflections["tarjet_1"].view(-1), ".-", color="red", label="tarjets_reflections_1")
        ax.set_title("Reflexion Tarjet 1", fontsize=20)
    elif i == 1:
        ax.plot(2*math.pi / physicsparams.k_2 * 1000, tarjet.tarjets_reflections["tarjet_2"].view(-1), ".-", color="green", label="tarjets_reflections_2")
        ax.set_title("Reflexion Tarjet 2", fontsize=20)
    elif i == 2:
        ax.plot(2*math.pi / physicsparams.k_3 * 1000, tarjet.tarjets_reflections["tarjet_3"].view(-1), ".-", color="blue", label="tarjets_reflections_3")
        ax.set_title("Reflexion Tarjet 3", fontsize=20)

    ax.set_xlabel("Wavelength (nm)", fontsize=16)
    ax.set_ylabel("Intensity", fontsize=16)
    ax.legend(fontsize=10)
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # Agregar líneas verticales punteadas
    for line_pos in vertical_lines:
        ax.axvline(x=line_pos, linestyle='--', color='gray')

# Mostrar la figura de reflexiones
plt.show()

In [ ]:
for seed in range(1):                                                # Generacion del comando for para las semillas (0, ->(cantidad de semillas aleatorias), 1)
  params.seed = seed

  torch.manual_seed(seed)
  glonet = GLOnet(params, physicsparams, tarjet)
  glonet.train()
  glonet.viz_training()
  print(f"iteration{seed + 1}")
  

In [ ]:
with torch.no_grad():
  params.k_test = 2 * math.pi / torch.linspace(0.3, 2.5, 50)
  params.theta_test = torch.linspace(0, math.pi/2.25, 50)
  (thicknesses, ref_index, result_mat) = glonet.evaluate(150, kvector = params.k_test,  inc_angles = params.theta_test, grayscale=True)
  reflex, transmix = MTMM_solver(params.condiciones, thicknesses, ref_index, physicsparams)
  for i in range(1, physicsparams.condiciones):
      reflex_key = f'reflexion_{i}'
      tarjet_key = f'tarjet_{i}'
      r = reflex[reflex_key]
      FoM_reflex = torch.pow(r - tarjet.tarjets_reflections[tarjet_key], 2).mean(dim=[1, 2, 3])  
  for i in range(1, physicsparams.condiciones):
      transmision_key = f'transmision_{i}'
      tarjet_key = f'tarjet_{i}'
      t = transmix[transmision_key]
      FoM_transmix = torch.pow(t - tarjet.tarjets_transmisions[tarjet_key], 2).mean(dim=[1, 2, 3])

  FoM_total =  FoM_reflex + FoM_transmix
  plt.figure(figsize = (10, 2))
  plt.subplot(131)
  plt.hist(FoM_total.cpu().detach().numpy(), alpha = 0.5)
  plt.xlabel(f"FoM (n' = {n_interna})", fontsize=18)
  plt.xticks(fontsize=14)
  plt.yticks(fontsize=14)

_, indices = torch.sort(FoM_total)
opt_idx = indices[0]

optimal_thicknesses = thicknesses[opt_idx]
optimal_ref_idx = ref_index[opt_idx]

print("Espesores:" , optimal_thicknesses)
print("Indices de refraccion",optimal_ref_idx)

optimal_reflections = {}
optimal_transmisions = {}

for i in range(1, params.condiciones + 1):
    reflex_key = f'reflexion_{i}'
    optimal_reflections[reflex_key] = reflex[reflex_key][opt_idx]
        
for i in range(1, params.condiciones + 1):
    transmision_key = f'transmision_{i}'
    optimal_transmisions[transmision_key] = transmix[transmision_key][opt_idx]

In [ ]:


xlim = (300, 800)
ylim = (-0.5, 1.5)
vertical_lines = [380, 495, 570, 750]
fig, axs = plt.subplots(1, 3, figsize=(13, 3))  # 1 fila, 3 columnas
fig.subplots_adjust(wspace=0.4)

for i in range(1, 4):
    # Gráfico de transmisión óptima
    axs[i-1].plot(2*math.pi / getattr(physicsparams, f'k_{i}') * 1000, optimal_transmisions[f'transmision_{i}'][:,0,0].detach().numpy(), "-", color="violet", label="Optimal Transmision")
    
    # Gráfico de transmisión de tarjeta
    if i == 1:
        tarjet_color = "red"
    elif i == 2:
        tarjet_color = "green"
    elif i == 3:
        tarjet_color = "blue"
    
    axs[i-1].plot(2*math.pi / getattr(physicsparams, f'k_{i}') * 1000, tarjet.tarjets_transmisions[f"tarjet_{i}"].view(-1), "-o", color=tarjet_color, label=f"Tarjet Transmision {i}", linewidth=0.8)
    
    axs[i-1].set_xlabel("Wavelength (nm)", fontsize=15)
    axs[i-1].set_ylabel("Transmision", fontsize=15)
    axs[i-1].legend(loc="best")
    axs[i-1].set_title(f"Transmision {i}", fontsize=20)
    
    # Establecer límites y agregar líneas verticales punteadas para transmisiones
    axs[i-1].set_xlim(xlim)
    axs[i-1].set_ylim(ylim)
    
    for line_pos in vertical_lines:
        axs[i-1].axvline(x=line_pos, linestyle='--', color='gray')

# Mostrar la figura de transmisiones
plt.show()

# Subplots para Reflexiones
fig, axs = plt.subplots(1, 3, figsize=(13, 3))  # 1 fila, 3 columnas
fig.subplots_adjust(wspace=0.4)

# Loop sobre los índices 1, 2, 3 para los subplots de reflexiones
for i in range(1, 4):
    # Gráfico de reflexión óptima
    axs[i-1].plot(2*math.pi / getattr(physicsparams, f'k_{i}') * 1000, optimal_reflections[f'reflexion_{i}'][:,0,0].detach().numpy(), "-", color="violet", label="Optimal Reflexion")
    
    # Gráfico de reflexión de tarjeta
    if i == 1:
        tarjet_color = "red"
    elif i == 2:
        tarjet_color = "green"
    elif i == 3:
        tarjet_color = "blue"
    
    axs[i-1].plot(2*math.pi / getattr(physicsparams, f'k_{i}') * 1000, tarjet.tarjets_reflections[f"tarjet_{i}"].view(-1), ".-", color=tarjet_color, label=f"Tarjet Reflexion {i}", markersize=2.5)
    
    axs[i-1].set_xlabel("Wavelength (nm)", fontsize=16)
    axs[i-1].set_ylabel("Reflection", fontsize=16)
    axs[i-1].legend(fontsize=10)
    axs[i-1].set_title(f"Reflexion {i}", fontsize=20)
    axs[i-1].tick_params(axis='both', which='major', labelsize=14)
    axs[i-1].set_xlim(xlim)
    axs[i-1].set_ylim(ylim)

    # Agregar líneas verticales punteadas para reflexiones
    for line_pos in vertical_lines:
        axs[i-1].axvline(x=line_pos, linestyle='--', color='gray')


# Mostrar la figura de reflexiones
plt.show()
print("Espesores:" , optimal_thicknesses.view(-1))
print("Indices de refraccion:",optimal_ref_idx.view(-1))

In [ ]:

(thicknesses, ref_index, result_mat) = glonet.evaluate(150, kvector = params.k_test,  inc_angles = params.theta_test, grayscale=True)
r = []
t = []
params.k_test = 2 * math.pi / torch.linspace(0.3, 0.9, 50)
params.theta_test = torch.linspace(0, math.pi/2.25, 50)

for i in range(3):  
    r_ , t_= TMM_solver(thicknesses , 
                            ref_index, 
                            getattr(physicsparams, f'n_top_{i + 1}'), 
                            getattr(physicsparams, f'n_bot_{i + 1}'),
                            params.k_test,
                            params.theta_test)
    r.append(r_)
    t.append(t_)

_ ,  index = torch.sort(FoM_total)
opt_inx =  index[0]


fig, axs = plt.subplots(1,3, figsize=(13, 3))
fig.subplots_adjust(wspace=0.4)
for i in range(3):
    im1 = axs[i].imshow(t[i][opt_inx, : , 0, :].detach().cpu().numpy(), origin='lower', aspect='auto', cmap='hot', extent=[0, 80, 300, 900])
    cbar1 = fig.colorbar(im1, ax=axs[i])
    cbar1.ax.tick_params(labelsize=14)
    axs[i].set_title(f'Transmision_{i}', fontsize=20)
    axs[i].set_xlabel('Incident angle (deg)', fontsize=16)
    axs[i].set_ylabel('Wavelength (nm)', fontsize=16)
    axs[i].tick_params(axis='both', which='major', labelsize=14)

fig, axs = plt.subplots(1,3, figsize=(13, 3))
fig.subplots_adjust(wspace=0.4)
for i in range(3):
    im1 = axs[i].imshow(r[i][opt_inx, : , 0, :].detach().cpu().numpy(), origin='lower', aspect='auto', cmap='hot', extent=[0, 80, 300, 900])
    cbar1 = fig.colorbar(im1, ax=axs[i])
    cbar1.ax.tick_params(labelsize=14)
    axs[i].set_title(f'Reflexion_{i}', fontsize=20)
    axs[i].set_xlabel('Incident angle (deg)', fontsize=16)
    axs[i].set_ylabel('Wavelength (nm)', fontsize=16)
    axs[i].tick_params(axis='both', which='major', labelsize=14)
    

# Vista que valor le doy a n_interna


In [ ]:

for n_value in range(20):
  
  n_interna = 1.2 + 0.2 * n_value / 20

  physicsparams.n_bot_1 = n_interna
  physicsparams.n_top_1 = 1
  physicsparams.k_1 = 370
  physicsparams.k_values.append(physicsparams.k_1)
  physicsparams.theta_1 = 45
  physicsparams.lambda_min_1 = 380
  physicsparams.lambda_max_1 = 750
  physicsparams.pol_1 = "TE"

  physicsparams.n_bot_2 = 1
  physicsparams.n_top_2 = n_interna
  physicsparams.k_2 = 190
  physicsparams.k_values.append(physicsparams.k_2)
  physicsparams.theta_2 = 45
  physicsparams.lambda_min_2 = 380
  physicsparams.lambda_max_2 = 570
  physicsparams.pol_2 = "TE"

  physicsparams.n_bot_3 = 1
  physicsparams.n_top_3 = n_interna
  physicsparams.k_3 = 115
  physicsparams.k_values.append(physicsparams.k_3)
  physicsparams.theta_3 = 0
  physicsparams.lambda_min_3 = 380
  physicsparams.lambda_max_3 = 495
  physicsparams.pol_3 = "TM"

  physicsparams.generate_physics_params()
  params.seed = n_value
  

  torch.manual_seed(n_value)
  
  glonet = GLOnet(params, physicsparams, tarjet)
  glonet.train()
  # glonet.viz_training()
  print(f"iteration{n_value + 1 }")
  print(f" n` = {n_interna}")

  with torch.no_grad():
    params.k_test = 2 * math.pi / torch.linspace(0.3, 2.5, 50)
    params.theta_test = torch.linspace(0, math.pi/2.25, 50)

    (thicknesses, ref_index, result_mat) = glonet.evaluate(150, kvector = params.k_test,  inc_angles = params.theta_test, grayscale=True)
    reflex, transmix = MTMM_solver(params.condiciones, thicknesses, ref_index, physicsparams)

    for i in range(1, physicsparams.condiciones):
        reflex_key = f'reflexion_{i}'
        tarjet_key = f'tarjet_{i}'
        r = reflex[reflex_key]
        FoM_reflex = torch.pow(r - tarjet.tarjets_reflections[tarjet_key], 2).mean(dim=[1, 2, 3])  


    for i in range(1, physicsparams.condiciones):
        transmision_key = f'transmision_{i}'
        tarjet_key = f'tarjet_{i}'
        t = transmix[transmision_key]
        FoM_transmix = torch.pow(t - tarjet.tarjets_transmisions[tarjet_key], 2).mean(dim=[1, 2, 3])

    FoM_total =  FoM_reflex + FoM_transmix
    plt.figure(figsize = (10, 2))
    plt.subplot(131)
    plt.hist(FoM_total.cpu().detach().numpy(), alpha = 0.5)
    plt.xlabel(f"FoM (n' = {n_interna})", fontsize=18)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)




# R y T absorbentes


In [ ]:
with torch.no_grad():
  num_devices = 6
  params.k_test = 2 * math.pi / torch.linspace(0.3, 2.5, 200)
  params.theta_test = torch.linspace(0, math.pi/2.25, 200)

(thicknesses, ref_index, result_mat) = glonet.evaluate(150, kvector = params.k_test,  inc_angles = params.theta_test, grayscale=True)
Reflexion , Transmision =  TMM_solver(thicknesses, ref_index, physicsparams.n_bot_1,  physicsparams.n_bot_1, params.k_test , params.theta_test, pol = 'TM')

(optimal_thicknesses, optimal_ref_idx, optimal_reflections, optimal_transmisions , optimal_materials) = glonet.optimizer_evaluate(num_devices, params.k_test, params.theta_test )

print(optimal_materials)
print(optimal_thicknesses)
print(optimal_ref_idx)

FoM =  glonet.FoM_evaluate(100, kvector = params.k_test,  inc_angles = params.theta_test, grayscale=True)
_ ,  index = torch.sort(FoM)
opt_inx =  index[0]

In [ ]:
FoM =  glonet.FoM_evaluate(100, kvector = params.k_test,  inc_angles = params.theta_test, grayscale=True)
_ ,  index = torch.sort(FoM)
opt_inx =  index[0]

fig, axs = plt.subplots(2, 2, figsize=(10, 8))
# Ajustar el espacio entre los subplots
fig.subplots_adjust(wspace=0.4, hspace=0.4)  # Ajusta estos valores para más o menos espacio

# Primer subplot para Reflexion
im1 = axs[0, 0].imshow(Reflexion[opt_inx, : , 0, :].detach().cpu().numpy(), origin='lower', aspect='auto', cmap='hot', extent=[0, 80, 400, 1100])
cbar1 = fig.colorbar(im1, ax=axs[0, 0])
cbar1.ax.tick_params(labelsize=14)
axs[0, 0].set_title('Reflexion', fontsize=20)
axs[0, 0].set_xlabel('Incident angle (deg)', fontsize=16)
axs[0, 0].set_ylabel('Wavelength (nm)', fontsize=16)
axs[0, 0].tick_params(axis='both', which='major', labelsize=14)

# Segundo subplot para Transmision
im2 = axs[0, 1].imshow(Transmision[opt_inx, : , 0, :].detach().cpu().numpy(), origin='lower', aspect='auto', cmap='hot', extent=[0, 80, 400, 1100])
cbar2 = fig.colorbar(im2, ax=axs[0, 1])
cbar2.ax.tick_params(labelsize=14)
axs[0, 1].set_title('Transmision', fontsize=20)
axs[0, 1].set_xlabel('Incident angle (deg)', fontsize=16)
axs[0, 1].set_ylabel('Wavelength (nm)', fontsize=16)
axs[0, 1].tick_params(axis='both', which='major', labelsize=14)

# Tercer subplot para Loss training
axs[1, 0].plot(glonet.loss_training)
axs[1, 0].set_title('Loss Training', fontsize=20)
axs[1, 0].set_ylabel('Loss', fontsize=18)
axs[1, 0].set_xlabel('Iterations', fontsize=18)
axs[1, 0].tick_params(axis='both', which='major', labelsize=14)

# Cuarto subplot para FoM histogram
axs[1, 1].hist(FoM.cpu().detach().numpy(), alpha=0.5)
axs[1, 1].set_xlabel('FoM (Figure of Merit)', fontsize=18)
axs[1, 1].tick_params(axis='both', which='major', labelsize=14)

In [ ]:

# Crear la figura y los subplots
fig, axs = plt.subplots(1, 4, figsize=(15, 3))
# Ajustar el espacio entre los subplots
fig.subplots_adjust(wspace=0.4)

# Primer subplot para Reflexion
im1 = axs[0].imshow(Reflexion[opt_inx, : , 0, :].detach().cpu().numpy(), origin='lower', aspect='auto', cmap='hot', extent=[0, 80, 400, 1100])
cbar1 = fig.colorbar(im1, ax=axs[0])
cbar1.ax.tick_params(labelsize=14)
axs[0].set_title('Reflexion', fontsize=20)
axs[0].set_xlabel('Incident angle (deg)', fontsize=16)
axs[0].set_ylabel('Wavelength (nm)', fontsize=16)
axs[0].tick_params(axis='both', which='major', labelsize=14)

# Segundo subplot para Transmision
im2 = axs[1].imshow(Transmision[opt_inx, : , 0, :].detach().cpu().numpy(), origin='lower', aspect='auto', cmap='hot', extent=[0, 80, 400, 1100])
cbar2 = fig.colorbar(im2, ax=axs[1])
cbar2.ax.tick_params(labelsize=14)
axs[1].set_title('Transmision', fontsize=20)
axs[1].set_xlabel('Incident angle (deg)', fontsize=16)
axs[1].set_ylabel('Wavelength (nm)', fontsize=16)
axs[1].tick_params(axis='both', which='major', labelsize=14)

# Tercer subplot para Loss training
axs[2].plot(glonet.loss_training)
axs[2].set_title('Loss Training', fontsize=20)
# axs[2].set_ylabel('Loss', fontsize=18)
axs[2].set_xlabel('Iterations', fontsize=18)
axs[2].tick_params(axis='both', which='major', labelsize=14)

# Cuarto subplot para FoM histogram
axs[3].hist(FoM.cpu().detach().numpy(), alpha=0.5)
axs[3].set_title('FoM Histogram', fontsize=20)
axs[3].set_xlabel('FoM (Figure of Merit)', fontsize=18)
axs[3].tick_params(axis='both', which='major', labelsize=14)

# Mostrar la figura
plt.show()


In [ ]:
plt.figure(figsize = (7, 5))
plt.plot(2*math.pi / physicsparams.k_1 * 1000, optimal_reflections['reflexion_1'][:,0,0].detach().numpy(), "-", color = "red", label = "Reflexion")
plt.plot(2*math.pi / physicsparams.k_1 * 1000,tarjet.tarjets_reflections["tarjet_1"].view(-1), "-o", label =  "Tarjet_Reflexion")

plt.plot(2*math.pi / physicsparams.k_1 * 1000, optimal_transmisions['transmision_1'][:,0,0].detach().numpy(), "-", color = "violet", label = "Transmision")
plt.plot(2*math.pi / physicsparams.k_1 * 1000,tarjet.tarjets_transmisions["tarjet_1"].view(-1), "-o", label = "Tarjet_Transmision")

plt.xlabel("Wavelength (nm)", fontsize = 15)
plt.ylabel("Reflection", fontsize = 15)
plt.legend(loc= "best")





In [ ]:
# Crear la figura y los subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 4))

# Primer subplot para Reflexion
ax1.plot(2 * math.pi / physicsparams.k_1 * 1000, optimal_reflections['reflexion_1'][:, 0, 0].detach().numpy(), "-", color="red", label="Reflexion")
ax1.plot(2 * math.pi / physicsparams.k_1 * 1000, tarjet.tarjets_reflections["tarjet_1"].view(-1), "-o", label="Tarjet_Reflexion")
ax1.set_xlabel("Wavelength (nm)", fontsize=15)
ax1.set_ylabel("Intesity", fontsize=15)
ax1.legend(loc="best")
ax1.set_title("Reflexion", fontsize=18)

# Segundo subplot para Transmision
ax2.plot(2 * math.pi / physicsparams.k_1 * 1000, optimal_transmisions['transmision_1'][:, 0, 0].detach().numpy(), "-", color="violet", label="Transmision")
ax2.plot(2 * math.pi / physicsparams.k_1 * 1000, tarjet.tarjets_transmisions["tarjet_1"].view(-1), "-o", label="Tarjet_Transmision")
ax2.set_xlabel("Wavelength (nm)", fontsize=15)
# ax2.set_ylabel("Transmission", fontsize=15)
ax2.legend(loc="best")
ax2.set_title("Transmision", fontsize=18)

# Mostrar la figura
plt.tight_layout()
plt.show()

In [ ]:
perdida_en_material = optimal_reflections['reflexion_1'][:,0,0].detach().numpy() + optimal_transmisions['transmision_1'][:,0,0].detach().numpy()
plt.plot(2*math.pi / physicsparams.k_1 * 1000,   perdida_en_material -1, "-", color = "black", label = "Perdida en material")

plt.axhline(y=0, color='b', linestyle='-', linewidth=2, label = "0-axis" )

plt.xlabel("Wavelength (nm)", fontsize = 15)
plt.ylabel("Loss", fontsize = 15)
plt.legend(loc= "best")

In [ ]:
plt.hist(FoM.cpu().detach().numpy())

# Ejemplo prioridades


In [ ]:
params.condiciones = 3
physicsparams = PhysicsParams(params.condiciones, user_define=True)

n_interna = 1.2

physicsparams.n_bot_1 = n_interna
physicsparams.n_top_1 = 1
physicsparams.k_1 = 370
physicsparams.k_values.append(physicsparams.k_1)
physicsparams.theta_1 = 45
physicsparams.lambda_min_1 = 380
physicsparams.lambda_max_1 = 750
physicsparams.pol_1 = "TE"

physicsparams.n_bot_2 = 1
physicsparams.n_top_2 = n_interna
physicsparams.k_2 = 190
physicsparams.k_values.append(physicsparams.k_2)
physicsparams.theta_2 = 45
physicsparams.lambda_min_2 = 380
physicsparams.lambda_max_2 = 570
physicsparams.pol_2 = "TE"

physicsparams.n_bot_3 = 1
physicsparams.n_top_3 = n_interna
physicsparams.k_3 = 400
physicsparams.k_values.append(physicsparams.k_3)
physicsparams.theta_3 = 0
physicsparams.lambda_min_3 = 380
physicsparams.lambda_max_3 = 495
physicsparams.pol_3 = "TE"


physicsparams.generate_physics_params()
physicsparams.view_attributes()

tarjet = Tarjet(params.condiciones, physicsparams.k_values, params.user_define)
tarjet.configure_targets()

if not params.user_define:
    params.matdatabase =  MatDatabase(params.materials)
    params.n_database = params.matdatabase.interp_wv(2 * math.pi/physicsparams.ks, params.materials, True)
    params.M_materials =  params.n_database.size(0)
else:
    pass


In [ ]:
tarjet.tarjets_transmisions["tarjet_1"].view(-1)[0:190 ] = 1
tarjet.tarjets_reflections["tarjet_1"].view(-1)[190:] = 1

tarjet.tarjets_transmisions["tarjet_2"].view(-1)[0:115 ] = 1
tarjet.tarjets_reflections["tarjet_2"].view(-1)[115:] = 1

tarjet.tarjets_transmisions["tarjet_3"].view(-1)[:] = 1
tarjet.tarjets_reflections["tarjet_3"].view(-1)[:] = 0

In [ ]:

for seed in range(1):                                                # Generacion del comando for para las semillas (0, ->(cantidad de semillas aleatorias), 1)
  params.seed = seed

  torch.manual_seed(seed)
  glonet = GLOnet(params, physicsparams, tarjet)
  glonet.train()
  glonet.viz_training()
  print(f"iteration{seed + 1}")

In [ ]:
with torch.no_grad():
  params.k_test = 2 * math.pi / torch.linspace(0.3, 2.5, 50)
  params.theta_test = torch.linspace(0, math.pi/2.25, 50)
  (thicknesses, ref_index, result_mat) = glonet.evaluate(150, kvector = params.k_test,  inc_angles = params.theta_test, grayscale=True)
  reflex, transmix = MTMM_solver(params.condiciones, thicknesses, ref_index, physicsparams)
  for i in range(1, physicsparams.condiciones):
      reflex_key = f'reflexion_{i}'
      tarjet_key = f'tarjet_{i}'
      r = reflex[reflex_key]
      FoM_reflex = torch.pow(r - tarjet.tarjets_reflections[tarjet_key], 2).mean(dim=[1, 2, 3])  
  for i in range(1, physicsparams.condiciones):
      transmision_key = f'transmision_{i}'
      tarjet_key = f'tarjet_{i}'
      t = transmix[transmision_key]
      FoM_transmix = torch.pow(t - tarjet.tarjets_transmisions[tarjet_key], 2).mean(dim=[1, 2, 3])

  FoM_total =  FoM_reflex + FoM_transmix
  plt.figure(figsize = (10, 2))
  plt.subplot(131)
  plt.hist(FoM_total.cpu().detach().numpy(), alpha = 0.5)
  plt.xlabel(f"FoM (n' = {n_interna})", fontsize=18)
  plt.xticks(fontsize=14)
  plt.yticks(fontsize=14)

_, indices = torch.sort(FoM_total)
opt_idx = indices[0]

optimal_thicknesses = thicknesses[opt_idx]
optimal_ref_idx = ref_index[opt_idx]

print("Espesores:" , optimal_thicknesses)
print("Indices de refraccion",optimal_ref_idx)

optimal_reflections = {}
optimal_transmisions = {}

for i in range(1, params.condiciones + 1):
    reflex_key = f'reflexion_{i}'
    optimal_reflections[reflex_key] = reflex[reflex_key][opt_idx]
        
for i in range(1, params.condiciones + 1):
    transmision_key = f'transmision_{i}'
    optimal_transmisions[transmision_key] = transmix[transmision_key][opt_idx]

In [ ]:


xlim = (300, 800)
ylim = (-0.5, 1.5)
vertical_lines = [380, 495, 570, 750]
fig, axs = plt.subplots(1, 3, figsize=(13, 3))  # 1 fila, 3 columnas
fig.subplots_adjust(wspace=0.4)

for i in range(1, 4):
    # Gráfico de transmisión óptima
    axs[i-1].plot(2*math.pi / getattr(physicsparams, f'k_{i}') * 1000, optimal_transmisions[f'transmision_{i}'][:,0,0].detach().numpy(), "-", color="violet", label="Optimal Transmision")
    
    # Gráfico de transmisión de tarjeta
    if i == 1:
        tarjet_color = "red"
    elif i == 2:
        tarjet_color = "green"
    elif i == 3:
        tarjet_color = "blue"
    
    axs[i-1].plot(2*math.pi / getattr(physicsparams, f'k_{i}') * 1000, tarjet.tarjets_transmisions[f"tarjet_{i}"].view(-1), "-o", color=tarjet_color, label=f"Tarjet Transmision {i}", linewidth=0.8)
    
    axs[i-1].set_xlabel("Wavelength (nm)", fontsize=15)
    axs[i-1].set_ylabel("Transmision", fontsize=15)
    axs[i-1].legend(loc="best")
    axs[i-1].set_title(f"Transmision {i}", fontsize=20)
    
    # Establecer límites y agregar líneas verticales punteadas para transmisiones
    axs[i-1].set_xlim(xlim)
    axs[i-1].set_ylim(ylim)
    
    for line_pos in vertical_lines:
        axs[i-1].axvline(x=line_pos, linestyle='--', color='gray')

# Mostrar la figura de transmisiones
plt.show()

# Subplots para Reflexiones
fig, axs = plt.subplots(1, 3, figsize=(13, 3))  # 1 fila, 3 columnas
fig.subplots_adjust(wspace=0.4)

# Loop sobre los índices 1, 2, 3 para los subplots de reflexiones
for i in range(1, 4):
    # Gráfico de reflexión óptima
    axs[i-1].plot(2*math.pi / getattr(physicsparams, f'k_{i}') * 1000, optimal_reflections[f'reflexion_{i}'][:,0,0].detach().numpy(), "-", color="violet", label="Optimal Reflexion")
    
    # Gráfico de reflexión de tarjeta
    if i == 1:
        tarjet_color = "red"
    elif i == 2:
        tarjet_color = "green"
    elif i == 3:
        tarjet_color = "blue"
    
    axs[i-1].plot(2*math.pi / getattr(physicsparams, f'k_{i}') * 1000, tarjet.tarjets_reflections[f"tarjet_{i}"].view(-1), ".-", color=tarjet_color, label=f"Tarjet Reflexion {i}", markersize=2.5)
    
    axs[i-1].set_xlabel("Wavelength (nm)", fontsize=16)
    axs[i-1].set_ylabel("Reflection", fontsize=16)
    axs[i-1].legend(fontsize=10)
    axs[i-1].set_title(f"Reflexion {i}", fontsize=20)
    axs[i-1].tick_params(axis='both', which='major', labelsize=14)
    axs[i-1].set_xlim(xlim)
    axs[i-1].set_ylim(ylim)

    # Agregar líneas verticales punteadas para reflexiones
    for line_pos in vertical_lines:
        axs[i-1].axvline(x=line_pos, linestyle='--', color='gray')


# Mostrar la figura de reflexiones
plt.show()
print("Espesores:" , optimal_thicknesses.view(-1))
print("Indices de refraccion:",optimal_ref_idx.view(-1))